# Предсказываем стоимость мед страховки

In [9]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

Загружаем данные
Собраны данные:

age: возраст
sex: пол
bmi: индекс массы тела
children: количество детей, охваченных медицинским страхованием / количество иждивенцев
smoker: курение
region: регион (northeast, southeast, southwest, northwest).
charges: индивидуальные медицинские расходы (его и хотим предсказать)

In [10]:
# Загрузите данные из файла insurance.csv в переменную df
df = pd.read_csv('medical_insurance.csv', sep=',')

In [11]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


Смотрим статистику, что нет пропусков и отсуствующих значений

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [13]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


Преобразуем строковые данные

In [14]:
# Заменяем пол и курение на числа
df['sex']=df['sex'].map({'male': 1, 'female': 0})
df['smoker']=df['smoker'].map({'yes': 1,'no': 0})

In [15]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,southwest,16884.92400
1,18,1,33.770,1,0,southeast,1725.55230
2,28,1,33.000,3,0,southeast,4449.46200
3,33,1,22.705,0,0,northwest,21984.47061
4,32,1,28.880,0,0,northwest,3866.85520


In [16]:
# Заменяем регион на набор отдельных колонок (is_southwest, is_southeast и тп)
df = pd.get_dummies(df, columns=['region'])
df.head()

,age,sex,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,16884.92400,False,False,False,True
1,18,1,33.770,1,0,1725.55230,False,False,True,False
2,28,1,33.000,3,0,4449.46200,False,False,True,False
3,33,1,22.705,0,0,21984.47061,False,True,False,False
4,32,1,28.880,0,0,3866.85520,False,True,False,False


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               1338 non-null   int64  
 1   sex               1338 non-null   int64  
 2   bmi               1338 non-null   float64
 3   children          1338 non-null   int64  
 4   smoker            1338 non-null   int64  
 5   charges           1338 non-null   float64
 6   region_northeast  1338 non-null   bool   
 7   region_northwest  1338 non-null   bool   
 8   region_southeast  1338 non-null   bool   
 9   region_southwest  1338 non-null   bool   
dtypes: bool(4), float64(2), int64(4)
memory usage: 68.1 KB


# Формируем признаки и целевую переменную

In [18]:
from sklearn.model_selection import train_test_split## Сформируйте признаки и целевую переменную

In [19]:
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'charges',
       'region_northeast', 'region_northwest', 'region_southeast',
       'region_southwest'],
      dtype='object')

In [20]:
x = df[['age', 'sex', 'children', 'smoker', 'charges', 'region_northeast', 'region_northwest', 'region_southeast', 'region_southwest']]
y = df['charges']

# Разделяем данные на выборку для обучения/проверки

In [21]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# Создаем и обучаем модель линейной регресии

In [22]:
from sklearn.linear_model import LinearRegression

In [23]:
lr = LinearRegression() 

In [24]:
lr.fit(x_train, y_train)

LinearRegression()

In [25]:
lr.coef_

array([ 1.18011072e-13, -8.42548253e-13, -1.83345322e-14,  1.18573699e-12,
        1.00000000e+00, -1.06802418e-13, -1.69360955e-14,  5.81826848e-14,
        6.55558202e-14])

# Получаем предсказание и оцениваем качество

In [26]:
lr.predict(x_test)

array([ 4571.41305, 21223.6758 ,  5972.378  , 39774.2763 , 48173.361  ,
        8068.185  , 13224.693  , 16138.76205,  4719.73655,  2709.1119 ,
       32108.66282,  2904.088  ,  3645.0894 ,  7729.64575,  2217.46915,
       23807.2406 ,  5383.536  ,  2217.6012 , 30259.99556,  8782.469  ,
        4564.19145,  2304.0022 ,  1719.4363 ,  1725.5523 , 12979.358  ,
        3994.1778 ,  2585.269  ,  3392.3652 , 10736.87075,  1261.442  ,
        6986.697  ,  7160.094  ,  2709.24395,  1621.3402 ,  6610.1097 ,
       13430.265  ,  2866.091  , 11272.33139,  6196.448  ,  2045.68525,
        4391.652  , 11879.10405,  4058.71245, 38792.6856 ,  3077.0955 ,
       11566.30055, 17878.90068, 13880.949  , 13224.05705,  1253.936  ,
        4399.731  ,  8556.907  ,  1731.677  , 12928.7911 , 10791.96   ,
        3410.324  , 16232.847  ,  3989.841  , 11326.71487, 42969.8527 ,
        4894.7533 ,  6402.29135,  6664.68595,  4527.18295,  8603.8234 ,
        4667.60765, 23065.4207 ,  4350.5144 ,  5934.3798 , 19673

In [27]:
from sklearn.metrics import mean_squared_error# Оцените качество, при помощи метода mean_squared_error для тестовой выборки

In [28]:
mean_squared_error(y_test, lr.predict(x_test))

1.5435174797765382e-23

In [29]:
mean_squared_error(y_train, lr.predict(x_train))

1.430935489792521e-23

In [30]:
df

,age,sex,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,16884.92400,False,False,False,True
1,18,1,33.770,1,0,1725.55230,False,False,True,False
2,28,1,33.000,3,0,4449.46200,False,False,True,False
3,33,1,22.705,0,0,21984.47061,False,True,False,False
4,32,1,28.880,0,0,3866.85520,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...
1333,50,1,30.970,3,0,10600.54830,False,True,False,False
1334,18,0,31.920,0,0,2205.98080,True,False,False,False
1335,18,0,36.850,0,0,1629.83350,False,False,True,False
1336,21,0,25.800,0,0,2007.94500,False,False,False,True


In [31]:
# Делаем предсказание для одного человека

In [32]:
# Заполняем данные по конкретному человеку
data = [{    
    "age": 20,
    "sex": 1,
    "children": 2,
    "smoker": 1,
    "charges": 3886.0,
    "region_northeast": False,
    "region_northwest": 0,
    "region_southeast": 1,
    "region_southwest": 0
}]

In [33]:
df_person = pd.DataFrame(data)
df_person.head()

,age,sex,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,20,1,2,1,3886.0,False,0,1,0


In [34]:
lr.predict(df_person)

array([3886.])